In [1]:
import numpy as np
import pandas as pd
# import scipy
from matplotlib import pylab, gridspec, pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
np.set_printoptions(edgeitems=25, linewidth=180)

In [2]:
df = pd.read_csv("data.csv", index_col=False)

In [3]:
df.head()

,arr_line,dest_line,arr_station,dest_station
0,Сокольническая линия,Сокольническая линия,Черкизовская,Преображенская площадь
1,Сокольническая линия,Сокольническая линия,Преображенская площадь,Сокольники
2,Сокольническая линия,Сокольническая линия,Сокольники,Красносельская
3,Сокольническая линия,Сокольническая линия,Красносельская,Комсомольская
4,Сокольническая линия,Сокольническая линия,Комсомольская,Красные ворота


In [4]:
df['transf'] = False
df["arr_line"] = " " + df["arr_line"]

In [5]:
df.loc[df["arr_line"] != df["dest_line"], "transf"] = True

In [6]:
df.head(10)

,arr_line,dest_line,arr_station,dest_station,transf
0,Сокольническая линия,Сокольническая линия,Черкизовская,Преображенская площадь,False
1,Сокольническая линия,Сокольническая линия,Преображенская площадь,Сокольники,False
2,Сокольническая линия,Сокольническая линия,Сокольники,Красносельская,False
3,Сокольническая линия,Сокольническая линия,Красносельская,Комсомольская,False
4,Сокольническая линия,Сокольническая линия,Комсомольская,Красные ворота,False
5,Сокольническая линия,Кольцевая линия,Комсомольская,Комсомольская,True
6,Сокольническая линия,Сокольническая линия,Красные ворота,Чистые пруды,False
7,Сокольническая линия,Сокольническая линия,Чистые пруды,Лубянка,False
8,Сокольническая линия,Калужско-Рижская линия,Чистые пруды,Тургеневская,True
9,Сокольническая линия,Люблинская линия,Чистые пруды,Сретенский бульвар,True


In [7]:
new_df = pd.DataFrame(columns=['from', 'to'])

In [8]:
new_df['from'] = df['arr_line'] + df['arr_station']
new_df['to'] = df['dest_line'] + df['dest_station']
new_df.head()

,from,to
0,Сокольническая линия Черкизовская,Сокольническая линия Преображенская площадь
1,Сокольническая линия Преображенская площадь,Сокольническая линия Сокольники
2,Сокольническая линия Сокольники,Сокольническая линия Красносельская
3,Сокольническая линия Красносельская,Сокольническая линия Комсомольская
4,Сокольническая линия Комсомольская,Сокольническая линия Красные ворота


In [9]:
stations = np.unique(new_df[['from', 'to']].values)
dict_stations = dict(zip(stations, range(stations.shape[0])))
dict_transform = dict(zip(range(stations.shape[0]), stations))

In [10]:
adj_matrix = np.zeros([stations.shape[0], stations.shape[0]])
adj_matrix.shape

(202, 202)

In [11]:
new_df['from'] = new_df['from'].map(dict_stations)
new_df['to'] = new_df['to'].map(dict_stations)
new_df.head()

,from,to
0,164,158
1,158,160
2,160,152
3,152,151
4,151,153


In [12]:
data_l_in = list(new_df['from'])
data_l_out = list(new_df['to'])
i , j = 0, 0
while i < len(data_l_in) - 1:
    adj_matrix[data_l_out[i]][data_l_in[j]] = adj_matrix[data_l_in[j]][data_l_out[i]] = 1
    j+=1
    i+=1
adj_matrix = adj_matrix.astype(int)

In [13]:
print(dict_transform[0], '\n',  adj_matrix[0])

 Арбатско-Покровская линия Арбатская 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]


In [14]:
def neighbors(station: int) -> list:
    list_neighbors = []
    for i in range(len(adj_matrix[station])):
        if adj_matrix[station][i] == 1:
            list_neighbors.append(dict_transform[i])
    return list_neighbors

In [15]:
neighbors(0)

[' Арбатско-Покровская линия Площадь Революции',
 ' Арбатско-Покровская линия Смоленская',
 ' Серпуховско-Тимирязевская линия Боровицкая',
 ' Сокольническая линия Библиотека имени Ленина',
 ' Филевская линия Александровский сад']

In [16]:
graph = np.copy(adj_matrix)

In [17]:
print(graph)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0

In [18]:
def dijkstra(graph: 'np.ndarray', s:int, t:int)->'np.ndarray':
    '''finding the shortest paht between s-vertex & t-vertex'''
    size = len(graph[0])
    path = []
    dist = [np.inf for _ in range(size)]
    u = [False for _ in range(size)]
    
    dist[s] = 0
    
    for i in range(size):
#         cur = 0
        min_ = np.inf
        for j in range(size):
            if u[j] == False and dist[j] <= min_:
                min_ = dist[j]
                cur = j
        
        u[cur] = True
        
        for j in range(size):
            if graph[cur][j] > 0 and u[j] == False:
                if dist[cur] + graph[cur][j] < dist[j]:
                    dist[j] = dist[cur] + graph[cur][j]
    
#     print(dist)
    return dist
    

In [19]:
dist = dijkstra(graph, 0, 1)

In [20]:
def restore_path(cur_stat, cur_stat_dist, keys):
    res = 'from{} to: \n'.format(keys[cur_stat])
    for i in range(len(cur_stat_dist)):
        res += '{} : {} \n'.format(keys[i], cur_stat_dist[i])

    print(res)

In [21]:
restore_path(0, dist, dict_transform)

from Арбатско-Покровская линия Арбатская to: 
 Арбатско-Покровская линия Арбатская : 0 
 Арбатско-Покровская линия Бауманская : 3 
 Арбатско-Покровская линия Волоколамская : 10 
 Арбатско-Покровская линия Измайловская : 7 
 Арбатско-Покровская линия Киевская : 2 
 Арбатско-Покровская линия Крылатское : 7 
 Арбатско-Покровская линия Кунцевская : 5 
 Арбатско-Покровская линия Курская : 2 
 Арбатско-Покровская линия Митино : 11 
 Арбатско-Покровская линия Молодёжная : 6 
 Арбатско-Покровская линия Мякинино : 9 
 Арбатско-Покровская линия Парк Победы : 3 
 Арбатско-Покровская линия Партизанская : 6 
 Арбатско-Покровская линия Первомайская : 8 
 Арбатско-Покровская линия Площадь Революции : 1 
 Арбатско-Покровская линия Пятницкое шоссе : 12 
 Арбатско-Покровская линия Семёновская : 5 
 Арбатско-Покровская линия Славянский бульвар : 4 
 Арбатско-Покровская линия Смоленская : 1 
 Арбатско-Покровская линия Строгино : 8 
 Арбатско-Покровская линия Щёлковская : 9 
 Арбатско-Покровская линия Элек